In [1]:
import json
import requests
import hashlib
import datetime
from uuid import uuid4
from urllib.parse import urlparse
from flask import Flask, jsonify, request

In [2]:
class LoopChain:
    
    def __init__(self):
        self.chain = []
        self.transactions = []
        self.addBlock(proof =1, prevHash = "0")
        self.nodes = set()
    
    def addBlock(self, proof, prevHash):
        block = {
            "index": len(self.chain),
            "timestamp": str(datetime.datetime.now()),
            "proof": proof,
            "prevHash": prevHash,
            "transactions": self.transactions
        }
        self.transactions = []
        self.chain.append(block)
        return block
    
    def getlastBlock(self):
        return self.chain[-1]
    
    def hashBlock(self, block):
        encodedBlock = json.dumps(block, sort_keys=True).encode()
        return hashlib.sha256(encodedBlock).hexdigest()
    
    def proofOfwork(self, prevProof):
        newProof = 1
        solved = False 
        
        while solved is False:
            hashProblem = hashlib.sha256(str(newProof ** 2 - prevProof **2).encode()).hexdigest()
            if hashProblem[:4] == "0000":
                solved = True
            else:
                newProof += 1
        return newProof
    
    def addNode(self, address):
        userIp = urlparse(address)
        self.nodes.add(userIp.netloc)
    
    def addTransaction(self, sender, receiver, amount):
        self.transactions.append({
            "sender": sender,
            "receiver": receiver,
            "amount": amount
        })
        lastBlock = self.getlastBlock()
        return lastBlock["index"] + 1
    
    def isChainValid(self, chain):
        previousBlock = chain[0]
        current_index = 1 
        
        while current_index < len(chain):
            currentBlock = chain[current_index]
            currentPrevHash = currentBlock["prevHash"]
            if currentPrevHash != self.hashBlock(previousBlock):
                return False
            currentProof = currentBlock["proof"]
            previousProof = previousBlock["proof"]
            hashProblem = hashlib.sha256(str(currentProof ** 2 - previousProof **2).encode()).hexdigest()
            if hashProblem[:4] != "0000":
                return False
            previousBlock = currentBlock
            current_index += 1
        return True
    
    def shouldReplaceChain(self):
        network = self.nodes
        longestChain = None
        currentLength = len(self.chain)
        
        for node in network:
            nodeResponse = requests.get(f"http://{node}/chain")
            if nodeResponse.status_code == 200:
                length = nodeResponse()["length"]
                chain = nodeResponse()["chain"]
                if length > currentLength:
                    longestChain = chain
                    currentLength = length
            if longestChain:
                self.chain = chain
                return True
        return False

In [3]:
loop = LoopChain()

In [4]:
app = Flask(__name__)

In [5]:
nodeAddress = str(uuid4()).replace("-","")

In [6]:
@app.route("/mine", methods = ["GET"])
def mine():
    previousBlock = loop.getlastBlock()
    prevousProof = previousBlock["proof"]
    proof = loop.proofOfwork(prevousProof)
    previousHash = loop.hashBlock(previousBlock)
    loop.addTransaction(sender = nodeAddress, receiver = "Me", amount = 10)
    newBlock = loop.addBlock(proof, previousHash)
    response = {
        "message":"Congrats you have mined a new block",
        "index": newBlock["index"],
        "proof": newBlock["proof"],
        "time": newBlock["timestamp"],
        "prevhash": newBlock["prevHash"],
        "transactions": newBlock["transactions"]
    }
    return jsonify(response), 200 

In [7]:
@app.route("/chain", methods = ["GET"])
def chain():
    response = {
        'chain': loop.chain,
        "length": len(loop.chain)
    }
    return jsonify(response), 200

In [8]:
@app.route("/valid", methods = ["GET"])
def valid():
    isValid = loop.isChainValid(loop.chain)
    if isValid:
        response = {"Valid Chain": "This chain is valid to be build upon"}
    else:
        response = {"Invalid": "This chain is invalid"}
    return jsonify(response), 200

In [9]:
@app.route("/transaction", methods = ["POST"])
def transaction():
    transactionJson = request.get_json()
    transacitonKeys = ["sender","receiver","amount"]
    if not all (key in transactionJson for key in transacitonKeys):
        return "Transaciton missing keys", 400
    index = loop.addTransaction(transactionJson["sender"], transactionJson["receiver"], transactionJson["amount"])
    response = {
        "success": f"the transaction will be added to the block index {index}"
    }
    return jsonify(response), 201
    

In [10]:
@app.route("/connect", methods = ["POST"])
def connect():
    nodeResponse = request.get_json()
    nodes = nodeResponse.get("nodes")
    if nodes is None:
        return "There are no nodes", 400
    for node in nodes:
        loop.addNode(node)
        print(loop.nodes)
    response = {
        "message": 'all nodes have been connected',
        "nodeList": list(loop.nodes)
    }
    return jsonify(response), 201

In [11]:
@app.route("/replace", methods = ["GET"])
def replace():
    shouldReplace = loop.shouldReplaceChain()
    if shouldReplace:
        response = {
            "message": "The length of the chain needed to be updates",
            "New Chain": loop.chain,
            "new length": len(loop.chain)
        }
    else:
        response = {
            "message": "The length of the chain did not need to be updated",
            "current Lengh": len(loop.chain)
        }
    return jsonify(response), 200

In [ ]:
app.run(host="0.0.0.0", port=3000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:3000/ (Press CTRL+C to quit)
127.0.0.1 - - [31/Dec/2021 12:31:50] "GET /chain HTTP/1.1" 200 -
127.0.0.1 - - [31/Dec/2021 12:31:54] "GET /mine HTTP/1.1" 200 -
127.0.0.1 - - [31/Dec/2021 12:31:55] "GET /mine HTTP/1.1" 200 -
127.0.0.1 - - [31/Dec/2021 12:31:55] "GET /mine HTTP/1.1" 200 -
127.0.0.1 - - [31/Dec/2021 12:31:55] "GET /mine HTTP/1.1" 200 -
127.0.0.1 - - [31/Dec/2021 12:31:56] "GET /mine HTTP/1.1" 200 -
127.0.0.1 - - [31/Dec/2021 12:31:59] "GET /chain HTTP/1.1" 200 -
127.0.0.1 - - [31/Dec/2021 12:32:11] "GET /valid HTTP/1.1" 200 -
127.0.0.1 - - [31/Dec/2021 12:33:00] "POST /connect HTTP/1.1" 201 -


{''}
{''}
